In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import pandas as pd 
import numpy as np
import plotly.io as pio
from scipy import stats 
offline.init_notebook_mode(connected=True)
from IPython.display import display
from math import ceil




In [ ]:
arr=!echo $LOGDIR
FOLDER=arr[0]

In [ ]:
#Place / at the end of path

#Use paramter0_run/ for IMT=100 and use parameter1_run/ for IMT=10
FOLDER="/zfs/dicelab/DigitalPlant/results/June_30_02_13_00_node1791_Partition/parameter1_run/"
# pd.set_option('display.max_columns', None)


In [ ]:
#FOLDER="./"
df = [{"name":"4 Partitions", "df":None, "color":'rgba(0, 0, 255, 0.75)'}, {"name":"8 Partitions", "df":None, "color":'rgba(255, 0, 0, 0.75)'}, {"name":"16 Partitions", "df":None, "color":'rgba(0, 155, 0, 0.75)'}, {"name":"32 Partitions", "df":None, "color":'rgba(0, 0, 0, 0.75)'}] 

params = ["connection_strings_4.list/", "connection_strings_8.list/", "connection_strings_16.list/", "connection_strings_32.list/"]

for i in range(4):
    frame = pd.read_csv(FOLDER+params[i]+'trial0/log0_latencies.csv', na_filter= False)
    frame['Receive Time'] = pd.to_datetime(frame['Receive Time'], unit='ms')
    frame['Send Time'] = pd.to_datetime(frame['Send Time'], unit='ms')
    frame['Latency'] = (frame['Receive Time'] - frame['Send Time']).dt.total_seconds() * 1000
    df[i]["df"] = frame
    
    
# df = pd.read_csv(FOLDER+'all.csv', na_filter= False)
# df = df.sort_values(['deviceNum','Message ID'])

# devices = df.groupby('deviceNum')
# print(devices)
# df['IMT'] =  np.where(df['Message ID']== 0, np.nan, df['Send Time']- df['Send Time'].shift(1))


display(df)

In [ ]:
!cat {FOLDER}parameter*

In [ ]:
for frame in df:
    print(frame["df"].groupby(['Status']).count())

# df.groupby(['Status']).count()

In [ ]:
#get rid of the connection message
PERCENT_TO_DROP = .05

for frame in df:
    num_messages = frame["df"].shape[0]
    #print(num_messages)
    messages_to_drop = int(ceil(num_messages*PERCENT_TO_DROP))
    #print(messages_to_drop)
    print("Original dataframe contained {} messages, with each device sending {}\nDropping {}%, or {} messages from each side".format(len(frame["df"]), num_messages, PERCENT_TO_DROP,messages_to_drop))
    frame["df"].drop(frame["df"][frame["df"]['Message ID'] < messages_to_drop].index, inplace=True) #0 1 2 3 4
    origSend = frame["df"].at[messages_to_drop, "Send Time"]
    frame["df"]["Relative Send Time"] = frame["df"]["Send Time"] - origSend
    frame["df"]['Relative Send Time'] = frame["df"]['Relative Send Time'].apply(lambda x: float(x.total_seconds())) 
    frame["df"].count()
    print(frame["df"])
    
    
# num_messages = df.groupby('deviceNum')['deviceNum'].size()[0]
# messages_to_drop = int(ceil(num_messages*PERCENT_TO_DROP))

# print("Original dataframe contained {} messages, with each device sending {}\nDropping {}%, or {} messages from each side".format(len(df), num_messages, PERCENT_TO_DROP,messages_to_drop))
# df.drop(df[df['Message ID'] >= (num_messages - messages_to_drop)].index, inplace=True)
# df.drop(df[df['Message ID'] < messages_to_drop].index, inplace=True) #0 1 2 3 4


# print()


In [ ]:
fig_avg_lat = go.Figure()

# for frame in df:
for i in range(4):
    frame = df[3-i]
    # fig = px.histogram(frame, x="Relative Send Time", y="Latency", histfunc='avg')
    
    bins = np.linspace(frame["df"]["Relative Send Time"].min(), frame["df"]["Relative Send Time"].max(), 28)
    groups = frame["df"].groupby(pd.cut(frame["df"]["Relative Send Time"], bins))
    
    # print(groups.mean().Latency)
    
    fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, mode='lines', name=frame["name"], marker_color=frame["color"]))
    
fig_avg_lat.update_layout(
    title = "Average Latency (ms) vs Time (s)",
    xaxis_title = "Time (s)",
    yaxis_title = "Average Latency (ms)"
)
    
fig_avg_lat.show()
    
#     fig.add_trace(go.Histogram(x=frame["Relative Send Time"], y=frame["Latency"], histfunc='avg', xbins=dict(
#                       start='0',
#                       end='280',
#                       size='5'), 
#                       autobinx=False))
#     # fig.add_trace(go.Histogram(x=))
#     fig.update_layout(barmode='overlay')
#     fig.update_traces(opacity=1)
    
# fig.show()

In [ ]:
for frame in df:
    print(frame["df"]['Latency'].describe())


In [ ]:
fig_box = go.Figure()

for frame in df:
    fig_box.add_trace(go.Box(y=frame["df"]["Latency"], quartilemethod="inclusive", name=frame["name"], marker_color=frame['color'], boxpoints="outliers"))
    
fig_box.update_layout(
    title = "Latency vs Intermessage Time",
    xaxis_title = "Intermessage Time",
    yaxis_title = "Latency (ms)"
)    

fig_box.show()

In [ ]:
fig_cfd = go.Figure()

for frame in df:
    totalMessages = len(frame["df"])
    #freq = frame.groupby(['Latency']).count()['count']  # the count of each latency group
    freq = frame["df"].groupby(['Latency']).size()
    rel = freq / totalMessages # Divide that by the total messages to get the relative count
    cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
    fig_cfd.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, mode='lines', name=frame["name"], marker_color=frame["color"]))

fig_cfd.update_layout(
    title = "CFD of Latency (ms)",
    xaxis_title = "Latency (ms)",
    yaxis_title = "Probability"
)
fig_cfd.show()

In [ ]:
fig_mes_lat = go.Figure()
for frame in df:
    fig_mes_lat.add_trace(go.Scatter(x=frame["df"]["Relative Send Time"], y=frame["df"]["Latency"], mode='lines', name=frame["name"], marker_color=frame["color"]))

fig_mes_lat.update_layout(
    title = "Message Latency (ms) vs Relative Send Time (s)",
    xaxis_title = "Relative Send Time (s)",
    yaxis_title = "Message Latency (ms)"
) 
    
fig_mes_lat.show()


# fig_ts = px.scatter(df, x="Receive Time", y="Latency", color="deviceNum", title='Time Received vs Latency (ms) Colored by Device', render_mode='webgl')
# fig_ts.update_traces(line=dict(width=1))
# fig_ts.update_layout(
#     showlegend=False,
#     yaxis_title="Latency (ms)")
# fig_ts.show()